In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import warnings

# Download the stopwords corpus
import nltk
nltk.download('stopwords')

# Define Arabic stopwords
stopwords_list = stopwords.words('arabic')

# Read the TSV file
data = pd.read_csv("merged_reviews_finalDS.tsv", delimiter='\t')

# Rename the columns if necessary
data.columns = ['text', 'label']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anfel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
unique_labels = data['label'].unique()
print("Unique labels:", unique_labels)

Unique labels: [ 1  0 -1]


In [5]:
data.head()

,text,label
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1


In [6]:
from nltk.corpus import stopwords
import re

# Define functions for text cleaning
def clean_text(text):
    # Remove non-Arabic characters
    arabic_text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', arabic_text).strip()
    return cleaned_text

def remove_stopwords(text):
     # Split text into words
    words = text.split()
    # Remove stopwords
    text_without_stopwords = [word for word in words if word not in stopwords_list]
    # Join words back into a sentence
    return ' '.join(text_without_stopwords)

# Apply text cleaning functions to the 'text' column
data['cleaned_text'] = data['text'].apply(clean_text)
data['cleaned_text'] = data['cleaned_text'].apply(remove_stopwords)


In [7]:
data.head()

,text,label,cleaned_text
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1,أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب...
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1,هادفة وقوية تنقلك صخب شوارع القاهرة الى هدوء ج...
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1,خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ...
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1,ياسات جلوريا جزء يتجزأ دبي فندق متكامل الخدمات...


In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Verify the sizes of the train and test sets
print("Train set size:", len(train_data))
print("Test set size:", len(test_data))


Train set size: 131819
Test set size: 32955


In [9]:
train_data.head()

,text,label,cleaned_text
60537,مرضي. سهولة تسجيل الوصول وكمان الخروج على السا...,0,مرضي سهولة تسجيل الوصول وكمان الخروج الساحه وا...
51208,بحب الشعر العامى جداااا. بس بحب يكون فية وزن. ...,0,بحب الشعر العامى جداااا بحب يكون فية وزن وده ا...
17965,جيد. هدوء المكانقربه من المطاعم.,1,جيد هدوء المكانقربه المطاعم
50474,هربت من قبرها جليلة. كيف؟. ومع من ؟. وماهي حقي...,0,هربت قبرها جليلة كيف؟ ومع ؟ وماهي حقيقة الهروب...
15842,سلطنة عمان . اعجبني الطاقم كان ودود وحسن الظيا...,1,سلطنة عمان اعجبني الطاقم ودود وحسن الظيافة الم...


In [10]:
label_counts = np.bincount(data['label'] + 1)  # Shift labels to start from 0
label_counts_dict = {-1: label_counts[0], 0: label_counts[1], 1: label_counts[2]}

print("Number of instances for each label:")
print(label_counts_dict)

Number of instances for each label:
{-1: 41941, 0: 45705, 1: 77128}


In [11]:
!pip install torch torchvision torchaudio


In [12]:
# Additional imports for mBERT integration
from transformers import BertTokenizer, BertForSequenceClassification

# Load mBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')



In [13]:
# Load pre-trained mBERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)  # 3 labels for negative, neutral, positive



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Fine-tune mBERT model on your preprocessed dataset (not shown here)


In [15]:
import torch
# Function to tokenize text for mBERT input
def tokenize_text(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,  # Adjust as needed
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

# Function to predict sentiment using the fine-tuned mBERT model
def predict_sentiment(text):
    inputs = tokenize_text(text)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1).tolist()[0]
        predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class, probabilities



In [16]:
!pip install torch torchvision torchaudio


In [17]:
data.head()

,text,label,cleaned_text
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1,أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب...
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1,هادفة وقوية تنقلك صخب شوارع القاهرة الى هدوء ج...
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1,خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ...
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1,ياسات جلوريا جزء يتجزأ دبي فندق متكامل الخدمات...


In [ ]:
# Iterate through the preprocessed dataset and predict sentiment for each review
sentiments = []
probabilities = []
for text in data['cleaned_text']:
    sentiment, probs = predict_sentiment(text)
    sentiments.append(sentiment)
    probabilities.append(probs)


In [ ]:
sentiments

In [ ]:
data['label']

In [ ]:

# Map sentiment labels back to -1, 0, 1
data['predicted_sentiment'] = [sentiment - 1 for sentiment in sentiments]


In [ ]:
accuracy = (data['label'] == data['predicted_sentiment']).mean()

In [ ]:
print("accuracy",accuracy)

In [ ]:
train_data.label

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from tqdm import tqdm

# Define your dataset class
class CustomDataset(Dataset):
    
    def __init__(self, texts, labels, tokenizer, max_length):
        
        self.texts = texts
        
        self.labels = labels
        
        self.tokenizer = tokenizer
        
        self.max_length = max_length
        
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx): 
        text = self.texts[idx]
        label = self.labels[idx]+1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
# Define your training and validation datasets
train_dataset = CustomDataset(train_data.reset_index(drop=True)['cleaned_text'], train_data.reset_index(drop=True)['label'], tokenizer, max_length=500)
valid_dataset = CustomDataset(test_data.reset_index(drop=True)['cleaned_text'], test_data.reset_index(drop=True)['label'], tokenizer, max_length=500)


In [ ]:

# Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)  


In [ ]:

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5


In [ ]:

# Define data loaders
train_loader = DataLoader(train_dataset)

valid_loader = DataLoader(valid_dataset)


In [ ]:

# Fine-tuning loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    valid_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            valid_loss += loss.item()
            predictions = torch.argmax(outputs.logits, dim=1)
            predicted_labels = predictions - 1
            correct_predictions += torch.sum(predicted_labels == labels).item()
            total_predictions += len(labels)
    accuracy = correct_predictions / total_predictions
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss / len(train_loader)}, Validation Loss: {valid_loss / len(valid_loader)}, Validation Accuracy: {accuracy}')


In [67]:
data


,text,label,cleaned_text,predicted_sentiment
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...,0
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1,أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب...,1
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1,هادفة وقوية تنقلك صخب شوارع القاهرة الى هدوء ج...,1
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1,خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ...,0
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1,ياسات جلوريا جزء يتجزأ دبي فندق متكامل الخدمات...,0
...,...,...,...,...
164769,وهلأ لوين؟ ....سؤال اتمنى أن أطرحه في مصر اعتق...,1,وهلأ لوين؟ سؤال اتمنى أطرحه مصر اعتقد كلماتي ا...,1
164770,قراصنة الكاريبي بحس أنثوي ثلاثي الأبعاد بعد غي...,1,قراصنة الكاريبي بحس أنثوي ثلاثي الأبعاد غياب د...,1
164771,قصة بسيطة ودراما واقعية إسلوب إخراجي يتسم بالح...,0,قصة بسيطة ودراما واقعية إسلوب إخراجي يتسم بالح...,0
164772,بزوغ الفجر بتتويج دموي هكذا طل علينا فيلم ملحم...,1,بزوغ الفجر بتتويج دموي طل علينا فيلم ملحمة الش...,0


In [6]:
# Remove duplicates if needed
data.drop_duplicates(inplace=True)

# Display the size of the dataset after preprocessing
print("After removing duplicates and preprocessing, the dataset contains {} entries.".format(len(data)))

After removing duplicates and preprocessing, the dataset contains 161660 entries.


In [7]:
data_stat = data
data_stat.head()

,text,label,cleaned_text
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1,أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب...
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1,هادفة وقوية تنقلك صخب شوارع القاهرة الى هدوء ج...
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1,خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ...
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1,ياسات جلوريا جزء يتجزأ دبي فندق متكامل الخدمات...


In [10]:
pip install emoji


     -------------------------------------- 421.5/421.5 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import emoji
from nltk.corpus import stopwords

# Function to calculate average word length
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return (sum(len(word) for word in words) / len(words))

# Function to count emojis
def emoji_counter(sentence):
    return emoji.emoji_count(sentence)



# Calculate word count, character count, average characters per word, stopwords count, and emoji count
data_stat['word_count'] = data_stat['cleaned_text'].apply(lambda x: len(str(x).split(" ")))
data_stat['char_count'] = data_stat['cleaned_text'].str.len()  # This includes spaces
data_stat['avg_char_per_word'] = data_stat['cleaned_text'].apply(lambda x: avg_word(x))
stop = stopwords.words('arabic')
data_stat['stopwords_count'] = data_stat['cleaned_text'].apply(lambda x: len([x for x in x.split() if x in stop]))
data_stat['emoji_count'] = data_stat['cleaned_text'].apply(lambda x: emoji_counter(x))


# Display the DataFrame head
print(data_stat.head())


                                                text  label  \
0  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...      1   
1  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...      1   
2  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...      1   
3  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...      1   
4  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...      1   

                                        cleaned_text  word_count  char_count  \
0  ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...           7          51   
1  أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب...          39         222   
2  هادفة وقوية تنقلك صخب شوارع القاهرة الى هدوء ج...          34         208   
3  خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ...          89         492   
4  ياسات جلوريا جزء يتجزأ دبي فندق متكامل الخدمات...          11          62   

   avg_char_per_word  stopwords_count  emoji_count  
0           6.428571                0            0  
1           4.717949              

In [12]:
data_stat

,text,label,cleaned_text,word_count,char_count,avg_char_per_word,stopwords_count,emoji_count
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...,7,51,6.428571,0,0
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1,أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب...,39,222,4.717949,0,0
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1,هادفة وقوية تنقلك صخب شوارع القاهرة الى هدوء ج...,34,208,5.147059,0,0
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1,خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ...,89,492,4.539326,0,0
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1,ياسات جلوريا جزء يتجزأ دبي فندق متكامل الخدمات...,11,62,4.727273,0,0
...,...,...,...,...,...,...,...,...
164769,وهلأ لوين؟ ....سؤال اتمنى أن أطرحه في مصر اعتق...,1,وهلأ لوين؟ سؤال اتمنى أطرحه مصر اعتقد كلماتي ا...,306,1859,5.078431,0,0
164770,قراصنة الكاريبي بحس أنثوي ثلاثي الأبعاد بعد غي...,1,قراصنة الكاريبي بحس أنثوي ثلاثي الأبعاد غياب د...,493,3046,5.180527,0,0
164771,قصة بسيطة ودراما واقعية إسلوب إخراجي يتسم بالح...,0,قصة بسيطة ودراما واقعية إسلوب إخراجي يتسم بالح...,218,1371,5.293578,0,0
164772,بزوغ الفجر بتتويج دموي هكذا طل علينا فيلم ملحم...,1,بزوغ الفجر بتتويج دموي طل علينا فيلم ملحمة الش...,447,2876,5.436242,0,0


In [15]:
pip install textblob

     -------------------------------------- 626.3/626.3 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install tashaphyne

     -------------------------------------- 251.5/251.5 kB 1.9 MB/s eta 0:00:00
     -------------------------------------- 126.4/126.4 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install dsaraby

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement dsaraby (from versions: none)
ERROR: No matching distribution found for dsaraby


In [35]:
import re
from nltk.corpus import stopwords
from textblob import TextBlob
from tashaphyne.stemming import ArabicLightStemmer
import pyarabic.araby as araby



# Define Arabic stopwords
stops = set(stopwords.words("arabic"))
stop_word_comp = {"،", "آض", "آمينَ", "آه", "آهاً", "آي", "أ", "أب", "أجل", "أجمع", "أخ", "أخذ", "أصبح", "أضحى", "أقبل", "أقل", "أكثر", "ألا", "أم", "أما", "أمامك", "أمامكَ", "أمسى", "أمّا", "أن", "أنا", "أنت", "أنتم", "أنتما", "أنتن", "أنتِ", "أنشأ", "أنّى", "أو", "أوشك", "أولئك", "أولئكم", "أولاء", "أولالك", "أوّهْ", "أي", "أيا", "أين", "أينما", "أيّ", "أَنَّ", "أََيُّ", "أُفٍّ", "إذ", "إذا", "إذاً", "إذما", "إذن", "إلى", "إليكم", "إليكما", "إليكنّ", "إليكَ", "إلَيْكَ", "إلّا", "إمّا", "إن", "إنّما", "إي", "إياك", "إياكم", "إياكما", "إياكن", "إيانا", "إياه", "إياها", "إياهم", "إياهما", "إياهن", "إياي", "إيهٍ", "إِنَّ", "ا", "ابتدأ", "اثر", "احد", "اخرى", "اخلولق", "اذا", "اربعة", "ارتدّ", "استحال", "اطار", "اعادة", "اعلنت", "اف", "اكثر", "اكد", "الألاء", "الألى", "الا", "الاخيرة", "الان", "الاول", "الاولى", "التى", "التي", "الثاني", "الثانية", "الذاتي", "الذى", "الذي", "الذين", "السابق", "الف", "اللائي", "اللاتي", "اللتان", "اللتيا", "اللتين", "اللذان", "اللذين", "اللواتي", "الماضي", "المقبل", "الوقت", "الى", "اليوم", "اما", "امام", "امس", "ان", "انبرى", "انقلب", "انه", "انها", "او", "اول", "اي", "ايار", "ايام", "ايضا", "ب", "بات", "باسم", "بان", "بخٍ", "برس", "بسبب", "بسّ", "بشكل", "بضع", "بطآن", "بعد", "بعض", "بك", "بكم", "بكما", "بكن", "بل", "بلى", "بما", "بماذا", "بمن", "بن", "بنا", "به"}


In [36]:
conda install arabic-transliterate


^C

Note: you may need to restart the kernel to use updated packages.


In [38]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anfel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [39]:
# Initialize Arabic stemmer
ArListem = ArabicLightStemmer()

# Function to stem Arabic words
def stem(text):
    zen = TextBlob(text)
    words = zen.words
    cleaned = []
    for w in words:
        cleaned.append(ArListem.light_stem(w))
    return " ".join(cleaned)

# Function to normalize Arabic text
def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)

# Function to remove stopwords
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stopwords_list and len(w) >= 2])

# Apply text preprocessing functions to the 'text' column
data_stat['cleaned_text'] = data_stat['cleaned_text'].apply(stem)
data_stat['cleaned_text'] = data_stat['cleaned_text'].apply(normalizeArabic)
data_stat['cleaned_text'] = data_stat['cleaned_text'].apply(remove_stop_words)

# Display the DataFrame head
print(data_stat.head())

                                                text  label  \
0  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...      1   
1  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...      1   
2  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...      1   
3  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...      1   
4  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...      1   

                                        cleaned_text  word_count  char_count  \
0  mmtaz nwaa ma alndaf walmwqa waltjhyz walshaty...           7          51   
1  hd sbab njah alاmarat اn kl shkhs fy hdhh aldw...          39         222   
2  hadf wqwy tnqlk mn skhb shwara alqahr alي hdwء...          34         208   
3  khlsna mbdءya ally mstny abhar zy alfyl alazrq...          89         492   
4  yasat jlwrya jzء la ytjzا mn dby fndq mtkaml a...          11          62   

   avg_char_per_word  stopwords_count  emoji_count  
0           6.428571                0            0  
1           4.717949              

In [40]:
data_stat

,text,label,cleaned_text,word_count,char_count,avg_char_per_word,stopwords_count,emoji_count
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1,mmtaz nwaa ma alndaf walmwqa waltjhyz walshaty...,7,51,6.428571,0,0
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1,hd sbab njah alاmarat اn kl shkhs fy hdhh aldw...,39,222,4.717949,0,0
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1,hadf wqwy tnqlk mn skhb shwara alqahr alي hdwء...,34,208,5.147059,0,0
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1,khlsna mbdءya ally mstny abhar zy alfyl alazrq...,89,492,4.539326,0,0
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1,yasat jlwrya jzء la ytjzا mn dby fndq mtkaml a...,11,62,4.727273,0,0
...,...,...,...,...,...,...,...,...
164769,وهلأ لوين؟ ....سؤال اتمنى أن أطرحه في مصر اعتق...,1,whlا lwyn؟ sءal atmnي اn trhh fy msr aatqd اn ...,306,1859,5.078431,0,0
164770,قراصنة الكاريبي بحس أنثوي ثلاثي الأبعاد بعد غي...,1,qrasn alkaryby bhs nthwy thlathy alاbaad bad g...,493,3046,5.180527,0,0
164771,قصة بسيطة ودراما واقعية إسلوب إخراجي يتسم بالح...,0,qs bsyt wdrama waqay اslwb اkhrajy ytsm balhsy...,218,1371,5.293578,0,0
164772,بزوغ الفجر بتتويج دموي هكذا طل علينا فيلم ملحم...,1,bzwgh alfjr bttwyj dmwy hkdha tl alyna fylm ml...,447,2876,5.436242,0,0


In [ ]:
# Save preprocessed data to a new file
data.to_csv("preprocessed_data.tsv", sep='\t', index=False)
